In [1]:
import torch
import flwr as fl
from torch.utils.data import Dataset, Subset
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor
from client import get_client_generator, weighted_average_accuracy
from dataset import partition_dataset
from util import seed_everything
from flwr.server.strategy import FedAvg

transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])
train_dataset = MNIST("./mnist", train=True, download=True, transform=transform)
val_dataset = MNIST("./mnist", train=False, transform=transform)

def complete_run(seed):
    seed_everything(seed)

    num_clients = 18
    train_datasets = partition_dataset(train_dataset, num_clients)
    val_datasets = partition_dataset(val_dataset, num_clients)
    train_dataloaders = [torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True) for dataset in train_datasets]
    val_dataloaders = [torch.utils.data.DataLoader(dataset, batch_size=16) for dataset in val_datasets]
    client_resources = None
    client_fn = get_client_generator(train_dataloaders, val_dataloaders)
    client_config = {
        "lr": 0.05,
        "epochs": 1,
    }
    strategy = FedAvg(
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        fraction_fit=0.5,
        fraction_evaluate=0.5,
        on_fit_config_fn=lambda _: client_config,
        on_evaluate_config_fn=lambda _: client_config,
        evaluate_metrics_aggregation_fn=weighted_average_accuracy,
    )
    hist = fl.simulation.start_simulation(
        client_fn=client_fn,
        seed_fn=seed_everything,
        seed=1234,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=1),
        client_resources=client_resources,
        strategy=strategy
    )
    return hist

/home/fredrik/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
run1 = complete_run(0)

INFO flwr 2023-03-08 12:41:26,428 | app.py:149 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)
2023-03-08 12:41:28,168	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-03-08 12:41:29,011 | app.py:183 | Flower VCE: Ray initialized with resources: {'CPU': 12.0, 'memory': 5737567028.0, 'object_store_memory': 2868783513.0, 'node:172.17.96.211': 1.0}
INFO flwr 2023-03-08 12:41:29,014 | server.py:96 | Initializing global parameters
INFO flwr 2023-03-08 12:41:29,015 | server.py:291 | Requesting initial parameters from one random client


initial seed 0 1056700
initial seed 1 3205573
initial seed 2 9521250
initial seed 3 3719574
initial seed 4 4003310
initial seed 5 2390659
initial seed 6 9109859
initial seed 7 7515682
initial seed 8 1530349
initial seed 9 1349656
initial seed 10 5369625
initial seed 11 8521829
initial seed 12 8208870
initial seed 13 1829687
Start client: 12
12 :  6390526


INFO flwr 2023-03-08 12:41:30,093 | server.py:296 | Received initial parameters from one random client
INFO flwr 2023-03-08 12:41:30,094 | server.py:99 | Evaluating initial parameters
INFO flwr 2023-03-08 12:41:30,094 | server.py:112 | FL starting
DEBUG flwr 2023-03-08 12:41:30,095 | server.py:232 | fit_round 1: strategy sampled 7 clients (out of 14)


7 :  5236437
1 :  8553978
0 :  6244022
12 :  8561430
9 :  5413177
11 :  3221233
10 :  8792835
(launch_and_fit pid=22598) Before train: -5.256171226501465


DEBUG flwr 2023-03-08 12:41:32,181 | server.py:246 | fit_round 1 received 7 results and 0 failures


(launch_and_fit pid=22599) Before train: -5.256171226501465
(launch_and_fit pid=22593) Before train: -5.256171226501465
(launch_and_fit pid=22594) Before train: -5.256171226501465
(launch_and_fit pid=22595) Before train: -5.256171226501465
(launch_and_fit pid=22592) Before train: -5.256171226501465


WARNING flwr 2023-03-08 12:41:32,216 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-08 12:41:32,217 | server.py:176 | evaluate_round 1: strategy sampled 7 clients (out of 14)


(launch_and_fit pid=22589) Before train: -5.256171226501465
5 :  5950579
10 :  5698400
9 :  2225242
7 :  9562916
11 :  5519838
12 :  9918466
2 :  7337212


DEBUG flwr 2023-03-08 12:41:33,736 | server.py:190 | evaluate_round 1 received 7 results and 0 failures
INFO flwr 2023-03-08 12:41:33,737 | server.py:155 | FL finished in 3.64185025200004
INFO flwr 2023-03-08 12:41:33,738 | app.py:209 | app_fit: losses_distributed [(1, 2.2934060990512313)]
INFO flwr 2023-03-08 12:41:33,738 | app.py:210 | app_fit: metrics_distributed {'accuracy': [(1, 0.1252250450090018)]}
INFO flwr 2023-03-08 12:41:33,738 | app.py:211 | app_fit: losses_centralized []
INFO flwr 2023-03-08 12:41:33,739 | app.py:212 | app_fit: metrics_centralized {}


[2.2895233731309905, 2.296617921994848, 2.2934601273523327, 2.2919983957328047, 2.2943637431168757, 2.2952073094557646, 2.2926728495351085]
2.2934060990512313


In [3]:
run2 = complete_run(0)

INFO flwr 2023-03-08 12:41:33,769 | app.py:149 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)
2023-03-08 12:41:37,361	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-03-08 12:41:38,270 | app.py:183 | Flower VCE: Ray initialized with resources: {'node:172.17.96.211': 1.0, 'object_store_memory': 2836874035.0, 'memory': 5673748071.0, 'CPU': 12.0}
INFO flwr 2023-03-08 12:41:38,272 | server.py:96 | Initializing global parameters
INFO flwr 2023-03-08 12:41:38,272 | server.py:291 | Requesting initial parameters from one random client


initial seed 0 9521250
initial seed 1 3719574
initial seed 2 4003310
initial seed 3 2390659
initial seed 4 9109859
initial seed 5 7515682
initial seed 6 1530349
initial seed 7 1349656
initial seed 8 5369625
initial seed 9 8521829
initial seed 10 8208870
initial seed 11 1829687
initial seed 12 5057437
initial seed 13 9248729
Start client: 12
12 :  3956067


INFO flwr 2023-03-08 12:41:39,374 | server.py:296 | Received initial parameters from one random client
INFO flwr 2023-03-08 12:41:39,375 | server.py:99 | Evaluating initial parameters
INFO flwr 2023-03-08 12:41:39,376 | server.py:112 | FL starting
DEBUG flwr 2023-03-08 12:41:39,377 | server.py:232 | fit_round 1: strategy sampled 7 clients (out of 14)


7 :  5413177
1 :  6496145
0 :  7337212
12 :  6461483
9 :  3221233
11 :  4145649
10 :  6390526
(launch_and_fit pid=23471) Before train: -10.768680572509766
(launch_and_fit pid=23468) Before train: -10.768680572509766
(launch_and_fit pid=23466) Before train: -10.768680572509766
(launch_and_fit pid=23463) Before train: -10.768680572509766
(launch_and_fit pid=23473) Before train: -10.768680572509766
(launch_and_fit pid=23472) Before train: -10.768680572509766


DEBUG flwr 2023-03-08 12:41:41,412 | server.py:246 | fit_round 1 received 7 results and 0 failures
WARNING flwr 2023-03-08 12:41:41,443 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-08 12:41:41,444 | server.py:176 | evaluate_round 1: strategy sampled 7 clients (out of 14)


(launch_and_fit pid=23464) Before train: -10.768680572509766
5 :  5236437
10 :  8561430
9 :  5519838
7 :  2225242
11 :  3466020
12 :  812637
2 :  2861705


DEBUG flwr 2023-03-08 12:41:42,962 | server.py:190 | evaluate_round 1 received 7 results and 0 failures
INFO flwr 2023-03-08 12:41:42,963 | server.py:155 | FL finished in 3.586099347999607
INFO flwr 2023-03-08 12:41:42,963 | app.py:209 | app_fit: losses_distributed [(1, 2.3025300265741624)]
INFO flwr 2023-03-08 12:41:42,964 | app.py:210 | app_fit: metrics_distributed {'accuracy': [(1, 0.10702140428085617)]}
INFO flwr 2023-03-08 12:41:42,964 | app.py:211 | app_fit: losses_centralized []
INFO flwr 2023-03-08 12:41:42,965 | app.py:212 | app_fit: metrics_centralized {}


[2.2990310011791584, 2.301396944275757, 2.306243292757777, 2.299610453159535, 2.300124534371854, 2.3056185612825453, 2.305680985884233]
2.3025300265741624


In [4]:
run1

History (loss, distributed):
	round 1: 2.2934060990512313
History (metrics, distributed):
{'accuracy': [(1, 0.1252250450090018)]}

In [5]:
run2

History (loss, distributed):
	round 1: 2.3025300265741624
History (metrics, distributed):
{'accuracy': [(1, 0.10702140428085617)]}